In [1]:
hip_lp_ckpt = '/niddk-data-central/leo_workspace/MoCA_result/LP/ckpt/Hip_20epochLP_blr_0.01_bs_256_input_size_[3, 100]/2025-04-27_02-37/checkpoint-best.pth'
wrist_lp_ckpt = '/niddk-data-central/leo_workspace/MoCA_result/LP/ckpt/Wrist_20epochblr_0.01_bs_256_input_size_[3, 100]/2025-04-26_20-00/checkpoint-best.pth'

In [2]:
import models_vit
import torch

hip_lp_model = models_vit.__dict__['vit_base_patch16'](
            img_size=[3,100], patch_size=[1, int(5)], 
            num_classes=2, in_chans=1, 
            global_pool=False)
hip_lp_model.head = torch.nn.Sequential(torch.nn.BatchNorm1d(768, affine=False, eps=1e-6), hip_lp_model.head)

wrist_lp_model = models_vit.__dict__['vit_base_patch16'](
            img_size=[3,100], patch_size=[1, int(5)], 
            num_classes=2, in_chans=1, 
            global_pool=False)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Hip-LP CM

In [3]:
hip_lp_checkpoint = torch.load(hip_lp_ckpt, map_location='cpu')['model']
if any(k.startswith("module.") for k in hip_lp_checkpoint.keys()):
    hip_lp_checkpoint = {k.replace("module.", ""): v for k, v in hip_lp_checkpoint.items()}
    
hip_lp_model.load_state_dict(hip_lp_checkpoint, strict=True)


<All keys matched successfully>

In [12]:
from util.datasets import iWatch_HDf5,collate_fn

def get_loader(data_path):
    dataset_val = iWatch_HDf5(data_path, set_type='val', transform=None)
    sampler_val = torch.utils.data.SequentialSampler(dataset_val)
    data_loader_val = torch.utils.data.DataLoader(
            dataset_val,
            sampler = sampler_val,
            batch_size=64,
            num_workers=2,
            pin_memory=True,
            drop_last=False,
            collate_fn = collate_fn)

    return data_loader_val

hip_loader = get_loader('/niddk-data-central/iWatch/pre_processed_seg/H')

In [13]:
from itertools import chain
from tqdm import tqdm
import numpy as np
from sklearn.metrics import balanced_accuracy_score

def mae_classification_pass(data_loader, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # ensures dropout etc. is disabled
    print('using', device)

    # Use flat Python lists
    preds_list = []
    targets_list = []

    with torch.no_grad():  # no gradient needed
        for batch in tqdm(data_loader):
            images = batch[0].to(device, non_blocking=True)
            target = batch[-1].to(device, non_blocking=True)

            output = model(images)
            pred = torch.argmax(output, dim=1)

            preds_list.extend(pred.cpu().tolist())
            targets_list.extend(target.cpu().tolist())

    balanced_acc = balanced_accuracy_score(targets_list, preds_list)
    return preds_list, targets_list, balanced_acc


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

pred_all, target_all, accuracy_all = mae_classification_pass(hip_loader,hip_lp_model)
trial_classification = f'hip lp, acc = {accuracy_all}'

cm_all = confusion_matrix(target_all, pred_all)

labels = ['Sitting','Non-Sitting']

plt.figure(figsize=(8, 6))
sns.heatmap(cm_all, annot=True, cmap='Blues', fmt='d', xticklabels=labels,
            yticklabels=labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title(f'Accuracy (train)={round(accuracy_all,3)} ({trial_classification})')
plt.show()



using cuda


  0%|          | 0/6170 [00:00<?, ?it/s]